In [0]:
output_dir = f'report/{mode}/{pretrained}'
download_folder = 'data/dataset'
corpus = 'assin-ptpt'

##### get file from mt-dnn_port repo
mode = 'mt-dnn_assin+tweetsent' #'st-dnn', 'mt-dnn_assin', 'mt-dnn_assin+tweetsent'
pretrained = 'bert-pt'  #'bert', 'bert-pt', 'mt-dnn'

rte_filepath = \
f'output/{mode}/{corpus}-rte/{pretrained}/{corpus}-rte_test_scores_4.json' if mode == 'st-dnn' \
else  f'output/{mode}/{pretrained}/{corpus}-rte_test_scores_4.json'

sts_filepath = \
f'output/{mode}/{corpus}-sts/{pretrained}/{corpus}-sts_test_scores_4.json' if mode == 'st-dnn' \
else  f'output/{mode}/{pretrained}/{corpus}-sts_test_scores_4.json'
########

### change to other filepath here
filepaths = {
    'rte': rte_filepath,
    'sts': sts_filepath,
}

Get dataset and requirement

In [15]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1jkk-0YM2S8MEezIbjduptU6tMXgHzPOs' -O assin2-blind-test.xml
!wget http://nilc.icmc.usp.br/assin/assin.tar.gz
!tar -xzf assin.tar.gz

!tar -xzf assin.tar.gz
%rm -rf assin.tar.gz

!pip install xmltodict

%mkdir $download_folder
%mv *.xml $download_folder

--2020-04-18 19:50:00--  https://docs.google.com/uc?export=download&id=1jkk-0YM2S8MEezIbjduptU6tMXgHzPOs
Resolving docs.google.com (docs.google.com)... 172.217.203.139, 172.217.203.100, 172.217.203.101, ...
Connecting to docs.google.com (docs.google.com)|172.217.203.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-3g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/s9o1to661nc0g96nqga41batqb5rc3vg/1587239400000/06286219297811011113/*/1jkk-0YM2S8MEezIbjduptU6tMXgHzPOs?e=download [following]
--2020-04-18 19:50:00--  https://doc-14-3g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/s9o1to661nc0g96nqga41batqb5rc3vg/1587239400000/06286219297811011113/*/1jkk-0YM2S8MEezIbjduptU6tMXgHzPOs?e=download
Resolving doc-14-3g-docs.googleusercontent.com (doc-14-3g-docs.googleusercontent.com)... 172.217.204.132, 2607:f8b0:400c:c15::84
Connecting to doc-14-3g-docs.googleusercontent.com (doc-14

Import xml files

In [0]:
import xmltodict
import json

tasks = ['rte', 'sts']
scores = dict()

for task in tasks:
    filepath = filepaths[task]
    with open(filepath) as f:
        scores[task] = json.load(f)

    goldfile = 'assin2-blind-test.xml' if corpus == 'assin2' else f'{corpus}-test.xml'
    with open(f'{download_folder}/{goldfile}') as f:
        xml = xmltodict.parse(f.read())

Insert scores

In [0]:
for idx, item in enumerate(xml['entailment-corpus']['pair']):
    uid = xml['entailment-corpus']['pair'][idx]['@id']
    pos = scores['rte']['uids'].index(uid)

    similarity = scores['sts']['scores'][pos]
    entailment_labels = ['Entailment','None','Paraphrase']
    entailment = entailment_labels[scores['rte']['predictions'][pos]]

    xml['entailment-corpus']['pair'][idx]['@similarity'] = round(similarity, 1)
    xml['entailment-corpus']['pair'][idx]['@entailment'] = entailment

Save XML

In [0]:
result = xmltodict.unparse(xml, pretty = True)
with open(f'{output_dir}/{corpus}-test.xml', 'w') as f:
    f.write(result)